In [2]:
#collapse-hide
#Author: Sandeep Pawar
#Version: 1.0
#Date Mar 27, 2020

import pandas as pd
import numpy as np
import itertools

#Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
plt.style.use('seaborn-white')
%matplotlib inline

#statistics libraries
import statsmodels.api as sm
import scipy
from scipy.stats import anderson
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import month_plot, seasonal_plot, plot_acf, plot_pacf, quarter_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
from statsmodels.stats.diagnostic import acorr_ljungbox as ljung
#from nimbusml.timeseries import SsaForecaster
from statsmodels.tsa.statespace.tools import diff as diff
import pmdarima as pm
from pmdarima import ARIMA, auto_arima
from scipy import signal
from scipy.stats import shapiro
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler


#library to use R in Python 
import rpy2
# from rpy2.robjects import pandas2ri
# pandas2ri.activate()
 


import warnings
warnings.filterwarnings("ignore")
np.random.seed(786)

In [3]:
print('Pandas:', pd.__version__)
print('Statsmodels:', sm.__version__)
print('Scipy:', scipy.__version__)
print('Rpy2:', rpy2.__version__)

Pandas: 1.2.1
Statsmodels: 0.11.0
Scipy: 1.4.1
Rpy2: 3.4.5


In [4]:
path = 'https://raw.githubusercontent.com/pawarbi/datasets/master/timeseries/ts_frenchretail.csv'

#Sales numbers are in thousands, so I am dividing by 1000 to make it easier to work with numbers, especially squared errors
data = pd.read_csv(path, parse_dates=True, index_col="Date").div(1_000)

data.index.freq='Q'

data.head()  

Sales
Date             
2012-03-31  362.0
2012-06-30  385.0
2012-09-30  432.0
2012-12-31  341.0
2013-03-31  382.0

In [8]:
train = data.iloc[:-6]
test = data.iloc[-6:]

h = 6
train_length = len(train)

print('train_length:',train_length, '\n test_length:', len(test) )

train_length: 18 
 test_length: 6


# EDA

Finding missing data and duplicated date

In [9]:
print('missing_data:', train.isna().sum())
print('unique dates: ', train.index.nunique())

missing_data: Sales    0
dtype: int64
unique dates:  18


cross tabulation

In [12]:
#Counting number of values for each quarter and Year. Columns are quarters.
#Here each qquarter and year has 1 value, thus no duplicates
pd.crosstab(index = train.index.year, columns = train.index.quarter)

col_0  1  2  3  4
row_0            
2012   1  1  1  1
2013   1  1  1  1
2014   1  1  1  1
2015   1  1  1  1
2016   1  1  0  0

In [24]:
#collapse-hide
#Create line chart for Training data. index is reset to use Date column
train_chart=alt.Chart(train.reset_index()).mark_line(point=True).encode(
    x='Date', 
    y='Sales', 
    tooltip=['Date', 'Sales'])

#Create Rolling mean. This centered rolling mean 
rolling_mean = alt.Chart(train.reset_index()).mark_trail(
    color='orange',
    size=1
).transform_window(
    rolling_mean='mean(Sales)',
    frame=[-4,4]
).encode(
    x='Date:T',
    y='rolling_mean:Q',
    size='Sales'
)

#Add data labels
text = train_chart.mark_text(
    align='left',
    baseline='top',
    dx=5  # Moves text to right so it doesn't appear on top of the bar
).encode(
    text='Sales:Q'
)

#Add zoom-in/out
scales = alt.selection_interval(bind='scales')

#Combine everything
(train_chart + rolling_mean +text).properties(
    width=600, 
    title="French Retail Sales & 4Q Rolling mean ( in '000)").add_selection(
    scales
)

alt.LayerChart(...)

In [16]:
import altair as alt
from vega_datasets import data

cars = data.cars.url

alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon:Q',
    y='Horsepower:Q',
    color='Origin:N'
)

alt.Chart(...)

In [20]:
l = []
for n in range(10000):
    if n%10 == 0 and (n//10) % 35 == 0 and len(str(n)) == 4:
        l.append(n)
        
        
print(len(l))
        

26
